In [2]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import numpy as np
import pickle

In [16]:
file = open('book3.txt')   #processed book
data = file.read()
data = data.replace('\ufeff', "")
data = data[1:-1]
print("first line: " , data[:100])
print("last line: ", data[-100:])
print("Total words in this: ",len(data.split()))

first line:  roject gutenberg's the adventures of sherlock holmes by arthur conan doyle this ebook is for the use
last line:  lapped the hat upon his head it came right over the forehead and settled upon the bridge of his nose
Total words in this:  52217


In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])   #see we passing a list here important 

pickle.dump(tokenizer, open('M2tokenizer.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]  #see here texts to seq returns a list 

# fit_on_texts Updates internal vocabulary based on a list of texts. 
# This method creates the vocabulary index based on word frequency. 
# So if you give it something like, "The cat sat on the mat." 
# It will create a dictionary s.t. word_index["the"] = 1; word_index["cat"] = 2

In [18]:
vocab_size = len(tokenizer.word_index) + 1  #number of unique words as the list is 0 based index add 1
print(vocab_size)

5834


In [19]:
X = []
Y = []
for i in range(0 , len(sequence_data)-4):
    word = []
    for j in range(i , i+3):   
        word.append(sequence_data[j])
    X.append(word)
    Y.append(sequence_data[i+3])


print(tokenizer.sequences_to_texts([X[0]]))
print(X[0])
print(tokenizer.sequences_to_texts([[Y[0]]]))
print(Y[0])

["roject gutenberg's the"]
[2894, 2895, 1]
['adventures']
1003


In [20]:
X = np.array(X)
Y = np.array(Y)
Y = to_categorical(Y, num_classes=vocab_size)
print(Y[0])
print(Y[0][1546])

[0. 0. 0. ... 0. 0. 0.]
0.0


In [21]:
model = Sequential([
    Embedding(vocab_size, 10, input_length=3),
    LSTM(1000, return_sequences=True),
    LSTM(1000),
    Dense(1000, activation="relu"),
    Dense(vocab_size, activation="softmax")
]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3, 10)             58340     
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 1000)           4044000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_3 (Dense)              (None, 5834)              5839834   
Total params: 18,947,174
Trainable params: 18,947,174
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
history = model.fit(X , Y , epochs=60, batch_size=64)

Epoch 1/60
823/823 [==============================] - 255s 306ms/step - loss: 6.4617 - accuracy: 0.0581
Epoch 2/60
823/823 [==============================] - 242s 294ms/step - loss: 5.9411 - accuracy: 0.0820
Epoch 3/60
823/823 [==============================] - 242s 294ms/step - loss: 5.5858 - accuracy: 0.1007
Epoch 4/60
823/823 [==============================] - 238s 289ms/step - loss: 5.3210 - accuracy: 0.1126
Epoch 5/60
823/823 [==============================] - 235s 285ms/step - loss: 5.0979 - accuracy: 0.1252
Epoch 6/60
823/823 [==============================] - 248s 301ms/step - loss: 4.8910 - accuracy: 0.1349
Epoch 7/60
823/823 [==============================] - 239s 291ms/step - loss: 4.6781 - accuracy: 0.1475
Epoch 8/60
823/823 [==============================] - 240s 292ms/step - loss: 4.4509 - accuracy: 0.1574
Epoch 9/60
823/823 [==============================] - 238s 289ms/step - loss: 4.1911 - accuracy: 0.1718
Epoch 10/60
823/823 [==============================] - 230s 279m

In [25]:
pickle.dump(history.history, open('M2history.pkl', 'wb'))

In [26]:
model.save('m2.h5')